In [1]:
import sys,os
project_root = os.path.dirname(os.getcwd())
sys.path.insert(0,project_root)

from src.data_handling.load_data import *
from src.data_handling.preprocessing import *
from src.models.model_setup import *
from src.trainer.training import *
from src.trainer.file_utils import *
from transformers import set_seed

c:\Users\Hector Auvinen\Documents\GitHub\adapter_experiments\adapter_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

seed_value = 42
set_seed(seed_value)

# supported tasks:
# commonsense_qa: csqa
# cosmos_qa: cosqa # should this be classification?
# 


In [3]:
task = "cosmos_qa"
model_name = "bert-base-uncased"
output_dir = "C:/Users/Hector Auvinen/Desktop/eval_results"

In [4]:
data = load_hf_dataset(task,debug=False)

tokenizer = get_tokenizer(model_name)

encode = get_encoding(task)

Casting the dataset: 100%|██████████| 2985/2985 [00:00<00:00, 104347.22 examples/s]


using bert tokenizer
getting encoding:
<function encode_cosmosqa at 0x000001B16568DDC0>


In [5]:
dataset = preprocess_dataset(data,encode,tokenizer)

Map:  28%|██▊       | 7000/25262 [01:06<02:54, 104.65 examples/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Map: 100%|██████████| 2985/2985

mapped
{'train': ['id', 'context', 'question', 'answer0', 'answer1', 'answer2', 'answer3', 'label', 'input_ids', 'attention_mask'], 'test': ['id', 'context', 'question', 'answer0', 'answer1', 'answer2', 'answer3', 'label', 'input_ids', 'attention_mask'], 'validation': ['id', 'context', 'question', 'answer0', 'answer1', 'answer2', 'answer3', 'label', 'input_ids', 'attention_mask']}


In [8]:
num_labels = get_label_count(dataset)
print("labels",num_labels)
model = setup_model(model_name,num_labels,dataset)

adapter_config = adapters.BnConfig(
                          output_adapter=True,
                          mh_adapter=False,
                          reduction_factor=2,
                          non_linearity="relu")

model = add_mc_adapter(task_name=task,model=model,num_labels=num_labels,adapter_config=adapter_config)

default_args = TrainingParameters(per_device_train_batch_size=4)
default_args.lr_scheduler_type = "linear"
#default_args.per_device_eval_batch_size = 4
print(default_args.per_device_train_batch_size)

labels 4


Some weights of BertAdapterModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


2


In [9]:
train_args = get_training_arguments(default_args)

trainer = get_trainer(train_args,dataset,model,early_stopping=3)

trainer.train()

OutOfMemoryError: CUDA out of memory. Tried to allocate 90.00 MiB. GPU 0 has a total capacty of 3.00 GiB of which 0 bytes is free. Of the allocated memory 2.12 GiB is allocated by PyTorch, and 16.08 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
eval_results = trainer.evaluate()